In [10]:
%matplotlib inline

Perturbative Gadgets
==========================================

Following the derivation from Jordan et al. (2012), starting from a target k-local Hamiltonian:  
    $$H^{comp} = \sigma_{1} \sigma_{2} \dots \sigma_{n} $$
Using the corresponding gadget Hamiltonian:  
    $$H^{gad} = H^{anc} + V $$
with $H^{anc} = \sum\limits_{1\leq i \leq j \leq k} \frac{1}{2}(\mathbb{I} - Z_{i}Z_{j}) $
and $V = \sum\limits_{j=1}^k c_{j} \sigma_{j}\otimes X_{j}$  
one obtains that the shifted effective Hamiltonian on the low-energy subspace of the gadget Hamiltonian acting on the +1 eigenspace if $X^{\otimes n}$ behaves like the computational Hamiltonian
    $$\tilde{H}_{eff}(H_+^{gad}, 2^n, f(\lambda)) = \frac{-k(-\lambda)^k}{(k-1)!} H^{comp} \otimes P_+ + \mathcal{O}(\lambda^{k+1}) $$  

Looking again at the example Hamiltonian used by Holmes et al.
    $$H_G = \bigotimes_{i=1}^n \sigma_i^z $$
which has $r=1$ and $k=n$. For the example of $n=4$ one obtains: 
    $$H^{gad} = H^{anc} + V 
    = (\mathbb{I} - Z_1^{(a)} Z_2^{(a)}) + (\mathbb{I} - Z_1^{(a)} Z_3^{(a)}) + (\mathbb{I} - Z_1^{(a)} Z_4^{(a)})
    + (\mathbb{I} - Z_2^{(a)} Z_3^{(a)}) + (\mathbb{I} - Z_2^{(a)} Z_4^{(a)}) + (\mathbb{I} - Z_3^{(a)} Z_4^{(a)}) 
    + Z_1^{(c)} \otimes X_1^{(a)} + Z_2^{(c)} \otimes X_2^{(a)} + Z_3^{(c)} \otimes X_3^{(a)} + Z_4^{(c)} \otimes X_4^{(a)}$$

First importing the relevant packages

In [11]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
import datetime

np.random.seed(42)

Setting some parameters at the beginning for better visibility

In [12]:
data_to_produce = 'variance vs qubits'
# data_to_produce = 'variance vs layers'

# General parameters:
num_samples = 200
layers_list = [1, 2, 5]         # [1, 2, 5, 10, 20, 50]
# If data_to_produce == 'variance vs qubits'
qubits_list = [2, 4, 6]               # [2, 3, 4, 5, 6]
lambda_scaling = 0.5                        # w.r.t. λ_max
# If data_to_produce == 'variance vs layers'
qubit_number = 2
locality = qubit_number
lambda_max = (locality - 1) / (4 * locality)
lambda_list = [2**(-p)*lambda_max for p in range(4)]

In [13]:
def hardware_efficient_ansatz(params):
    """A random variational quantum circuit based on the hardware efficient ansatz. 
    There are no measurements and it is to be used within the global or local circuits

    Args:
        params (array[array[float]]): array of parameters of dimension (num_layers, num_qubits) containing the rotation angles

    Returns:
        not sure... see pennylane documentation
    """

    # Relevant parameters
    assert(len(np.shape(params)) == 2)      # check proper dimensions of params
    num_layers = np.shape(params)[0]        # np.shape(params) = (num_layers, num_qubits)
    num_qubits = np.shape(params)[1]
    
    # Generating the gate sequence from randomly applying RX, RY or RZ with the corresponding rotation angle
    gate_set = [qml.RX, qml.RY, qml.RZ]
    random_gate_sequence = [[np.random.choice(gate_set) for _ in range(num_qubits)] for _ in range(num_layers)]

    # Initial rotations on all qubits
    for i in range(num_qubits):             # rotate all qubits
        qml.RY(np.pi / 4, wires=i)          # "to prevent X, Y , or Z from being an especially preferential 
                                            # direction with respect to gradients."

    # Repeating a layer structure
    for l in range(num_layers):
        # Single random gate layer (single qubit rotations)
        for i in range(num_qubits):
            random_gate_sequence[l][i](params[l][i], wires=i)
        # Nearest neighbour controlled phase gates
        if num_qubits > 1:                          # no entangling gates if using a single qubit
            qml.broadcast(qml.CZ, wires=range(num_qubits), pattern="ring")



In [14]:
def global_circuit(params):
    assert(len(np.shape(params)) == 2)      # check proper dimensions of params
    num_qubits = np.shape(params)[1]        # np.shape(params) = (num_layers, num_qubits)

    hardware_efficient_ansatz(params)
    # Objective operator H = Z_1 Z_2 ... Z_n
    H = qml.PauliZ(0)
    for qubit in range(num_qubits-1):
        H = H @ qml.PauliZ(qubit + 1)
    return qml.expval(H)

def local_circuit(params):
    hardware_efficient_ansatz(params)
    # Objective operator H = Z_1 Z_2
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))
 
def gadget2_circuit(params, term, target_qubits):
    assert(len(np.shape(params)) == 2)
    computational_qubits = int(np.shape(params)[1] / 2)

    hardware_efficient_ansatz(params)

    # creating the "unperturbed Hamiltonian": Hanc
    if term == 'ancillary':
        # terms of the form I(aa)-Z(a)Z(a)
        term = qml.Identity(target_qubits[0]) @ qml.Identity(target_qubits[1]) - qml.PauliZ(target_qubits[0]) @ qml.PauliZ(target_qubits[1])
    elif term == 'coupling':
        # terms of the form Z(c)X(a)
        term = qml.PauliZ(target_qubits[0]) @ qml.PauliX(computational_qubits+target_qubits[0])
    return qml.expval(term)

def gadget3_circuit(params, term, target_qubits):
    assert len(np.shape(params)) == 2
    computational_qubits = int(np.shape(params)[1] * 2 / 3)

    hardware_efficient_ansatz(params)

    # creating the "unperturbed Hamiltonian": Hanc
    if term == 'ancillary':
        # terms of the form I(aa)-Z(a)Z(a)
        term = qml.Identity(target_qubits[0]) @ qml.Identity(target_qubits[1]) - qml.PauliZ(target_qubits[0]) @ qml.PauliZ(target_qubits[1])
    elif term == 'coupling':
        # terms of the form Z(c)Z(c)X(a)
        term = qml.PauliZ(target_qubits[0]) @ qml.PauliZ(target_qubits[1]) @ qml.PauliX(computational_qubits + target_qubits[2])
    return qml.expval(term)

In [15]:
def cost_function(qnode, params, circuit_type, num_qubits=0, lam=1):
    if circuit_type == gadget2_circuit:
        assert(num_qubits!=0)
        # Objective operator H = Hanc + V
        computational_qubits = num_qubits
        ancillary_qubits = computational_qubits
        total_qubits = ancillary_qubits + computational_qubits
        expval_terms = []
        # creating the "unperturbed Hamiltonian"
        # acting on the ancillary qubits only
        for first_qubit in range(computational_qubits, total_qubits):
            for second_qubit in range(first_qubit+1, total_qubits):
                expval_terms.append(qnode(params, 'ancillary', [first_qubit, second_qubit]))
        # creating the perturbation part of the Hamiltonian
        # acting on both ancillary and target qubits with the same index
        for qubit in range(computational_qubits):
            expval_terms.append(qnode(params, 'coupling', [qubit]))
        
        return np.sum(expval_terms[:-num_qubits]) + lam * np.sum(expval_terms[-num_qubits:])
    elif circuit_type == gadget3_circuit:
        assert num_qubits != 0
        assert num_qubits % 2 == 0, "3-local gadget decomposition only implemented for even qubit numbers"
        # Objective operator H = Hanc + V
        computational_qubits = num_qubits
        ancillary_qubits = int(computational_qubits  / 2)
        total_qubits = ancillary_qubits + computational_qubits
        expval_terms = []
        # creating the "unperturbed Hamiltonian"
        # acting on the ancillary qubits only
        for first_qubit in range(computational_qubits, total_qubits):
            for second_qubit in range(first_qubit+1, computational_qubits+ancillary_qubits):
                expval_terms.append(qnode(params, 'ancillary', [first_qubit, second_qubit]))
                print(expval_terms[-1])
        # creating the perturbation part of the Hamiltonian
        # acting on both ancillary and target qubits
        for qubit in range(ancillary_qubits):
            expval_terms.append(qnode(params, 'coupling', [2 * qubit, 2 * qubit + 1, qubit]))
            print(expval_terms[-1])
        
        return np.sum(expval_terms[:-num_qubits]) + lam * np.sum(expval_terms[-num_qubits:])
    else:
        return qnode(params)                # the cost function is the expectation value

In [16]:
def generate_gradients_vs_qubits(layer_list, qubit_list, circuit):
    with open('../../results/data/{}_{}_{}qubits_{}layers_{}lambda_{}samples.dat'
              .format(datetime.datetime.now().strftime("%y%m%d"), circuit.__name__, 
                      qubit_list[-1], layer_list[-1], lambda_scaling, num_samples), 'w') as of:
        of.write('# layers\t# qubits\tgradients')

        for num_layers in layer_list:
            print(num_layers, " layers")

            for num_qubits in qubit_list:
                grad_vals = []
                print(num_qubits, " qubits")

                # write a new line
                of.write('\n{}\t{}'.format(num_layers, num_qubits))

                for _ in range(num_samples):
                    if circuit == gadget2_circuit:
                        # Generating the random values for the rotations
                        params = np.random.uniform(0, np.pi, size=(num_layers, 2*num_qubits))
                        dev = qml.device("default.qubit", wires=range(2*num_qubits))    # /!\ only for r=1, k=n, k'=2
                    elif circuit == gadget3_circuit:
                        assert num_qubits % 2 == 0, "3-local gadget decomposition only implemented for even qubit numbers"
                        params = np.random.uniform(0, np.pi, size=(num_layers, int(1.5*num_qubits)))
                        dev = qml.device("default.qubit", wires=range(int(1.5*num_qubits)))    # /!\ only for r=1, k=n, k'=3
                    else:
                        params = np.random.uniform(0, np.pi, size=(num_layers, num_qubits))
                        dev = qml.device("default.qubit", wires=range(num_qubits))
                    qcircuit = qml.QNode(circuit, dev)

                    # Calculating the value of lambda to use
                    k = num_qubits
                    lambda_max = (k - 1) / (4 * k)
                    lambda_value = lambda_scaling * lambda_max

                    # Calculating the gradients of the cost function w.r.t the parameters
                    gradient = qml.grad(cost_function)(qcircuit, params, circuit, num_qubits, lambda_value)

                    # Write each newly calculated value (innefficient?)
                    of.write('\t{}'.format(gradient[0][0]))
                    
                # End file with one last line-break
                of.write('\n')


In [17]:
def generate_gradients_vs_layers(layer_list, lambda_list, qubit_number, circuit):
    assert(np.size(qubit_number) == 1)
    with open('../../results/data/{}_{}_{}qubits_{}layers_{}samples.dat'
              .format(datetime.datetime.now().strftime("%y%m%d"), circuit.__name__, 
                      qubit_number, layer_list[-1], num_samples), 'w') as of:
        of.write('# layers\tlambda\tgradients for {} qubits'.format(qubit_number))
        num_qubits = qubit_number

        for num_layers in layer_list:
            print(num_layers, " layers")

            for lam in lambda_list:

                # write a new line
                of.write('\n{}\t{}'.format(num_layers, lam))

                for _ in range(num_samples):
                    if circuit != gadget2_circuit:
                        dev = qml.device("default.qubit", wires=range(num_qubits))
                    else:
                        dev = qml.device("default.qubit", wires=range(2*num_qubits))    # /!\ only for r=1, k=n, k'=2
                    qcircuit = qml.QNode(circuit, dev)

                    params = np.random.uniform(0, np.pi, size=(num_layers, num_qubits))

                    gradient = qml.grad(cost_function)(qcircuit, params, circuit==gadget2_circuit, num_qubits, lam=lam)

                    # Write each newly calculated value (innefficient?)
                    of.write('\t{}'.format(gradient[0][0]))
                    
                # End file with one last line-break
                of.write('\n')


In [18]:
if data_to_produce == 'variance vs qubits':
    generate_gradients_vs_qubits(layers_list, qubits_list, global_circuit)
    generate_gradients_vs_qubits(layers_list, qubits_list, local_circuit)
    generate_gradients_vs_qubits(layers_list, qubits_list, gadget2_circuit)
    generate_gradients_vs_qubits(layers_list, qubits_list, gadget3_circuit)
elif data_to_produce == 'variance vs layers':
    generate_gradients_vs_layers(layers_list, lambda_list, qubit_number, gadget2_circuit)

1  layers
2  qubits
4  qubits
6  qubits
2  layers
2  qubits
4  qubits
6  qubits
5  layers
2  qubits
4  qubits
6  qubits
1  layers
2  qubits
4  qubits
6  qubits
2  layers
2  qubits
4  qubits
6  qubits
5  layers
2  qubits
4  qubits
6  qubits
1  layers
2  qubits
4  qubits
6  qubits
2  layers
2  qubits
4  qubits
6  qubits
5  layers
2  qubits
4  qubits
6  qubits
1  layers
2  qubits
Autograd ArrayBox with value 0.3076391042979696
Autograd ArrayBox with value 0.7071067811865476
Autograd ArrayBox with value -0.5995006394064104
Autograd ArrayBox with value 0.7071067811865472
Autograd ArrayBox with value -0.17101153751989065
Autograd ArrayBox with value 0.17532319595555795
Autograd ArrayBox with value 0.05040873988100147
Autograd ArrayBox with value 0.15996082149870627
Autograd ArrayBox with value 0.7071067811865476
Autograd ArrayBox with value 0.8071814616335562
Autograd ArrayBox with value -0.7048575485027078
Autograd ArrayBox with value 0.7071067811865476
Autograd ArrayBox with value 0.707106